# Intro

LLMs are becoming increasingly integrated into development workflows,
creating demand for frameworks that make AI-based application development accessible.
In the JVM and Kotlin ecosystem, Spring is exactly such a framework.
Yes, that same Spring that has been around for over 20 years!

Spring AI is a new addition that simplifies AI integration into your Kotlin code.
Spring AI provides:

* A unified API for working with various AI models and LLM providers (OpenAI, Anthropic, Ollama, etc.)
* Components for prompt processing and context management
* Built-in capabilities for vector stores and RAG applications

In this series of posts, we'll explore the core features of Spring AI.
Following programming tradition,
our first post will be a "Hello, world!" of sorts:
we'll connect to an LLM provider and ask it to tell us a joke.

First, let's add the necessary dependencies.
> [!NOTE] We'll be working with Claude models from Anthropic,
> but we'll include commented code for OpenAI as well.
> This will help you understand the general API approach regardless of which model you choose.

In [1]:
%useLatestDescriptors
%use spring-ai-anthropic
//%use spring-ai-openai

To use the model, we need to provide an API key.

You can obtain this API key from
[console.anthropic.com](https://console.anthropic.com/settings/keys)
for Anthropic models or from
[platform.openai.com](https://platform.openai.com/api-keys)
for OpenAI models.

Then add the generated API key to your environment variables:

[MacOS/Linux]
```bash
export ANTHROPIC_API_KEY=<INSERT KEY HERE> # for Anthropic
export OPENAI_API_KEY=<INSERT KEY HERE> # for OpenAI

```

[Windows]
```shell
set ANTHROPIC_API_KEY=<INSERT KEY HERE> # for Anthropic
set OPENAI_API_KEY=<INSERT KEY HERE> # for OpenAI
```

Let's retrieve the API key from environment variables:

In [2]:
val apiKey = System.getenv("ANTHROPIC_API_KEY") ?: "YOUR_ANTHROPIC_API_KEY"
//val apiKey = System.getenv("OPENAI_API_KEY") ?: "YOUR_OPENAI_API_KEY"

First, let's create a low-level API client for our chosen provider.
This only requires the API key:

In [3]:
val anthropicApi = AnthropicApi.builder().apiKey(apiKey).build()
//val openAiApi = OpenAiApi.builder().apiKey(apiKey).build()

Now that we have our client, we can make a simple model call:

In [4]:
val anthropicMessage = AnthropicApi.AnthropicMessage(
    listOf(AnthropicApi.ContentBlock("Tell me a joke")), AnthropicApi.Role.USER
)
//val openAiMessage = OpenAiApi.ChatCompletionMessage("Tell me a joke", OpenAiApi.ChatCompletionMessage.Role.USER)


anthropicApi.chatCompletionEntity(
    AnthropicApi.ChatCompletionRequest(
        AnthropicApi.ChatModel.CLAUDE_SONNET_4_0.value,
        listOf(anthropicMessage), null, 100, 0.8, false
    )
)
//openAiApi.chatCompletionEntity(
//    OpenAiApi.ChatCompletionRequest(listOf(openAiMessage), OpenAiApi.ChatModel.CHATGPT_4_O_LATEST.value, 0.8, false)
//)

<200 OK OK,ChatCompletionResponse[id=msg_01PByqU55f2W72YbZ5WgcZLA, type=message, role=ASSISTANT, content=[ContentBlock[type=TEXT, source=null, text=Why don't scientists trust atoms?

Because they make up everything!, index=null, id=null, name=null, input=null, toolUseId=null, content=null, signature=null, thinking=null, data=null, cacheControl=null, title=null, context=null, citations=null]], model=claude-sonnet-4-20250514, stopReason=end_turn, stopSequence=null, usage=Usage[inputTokens=11, outputTokens=17, cacheCreationInputTokens=0, cacheReadInputTokens=0]],[:status:"200", anthropic-organization-id:"ea22ac71-22a7-43cc-9c6a-0a6a2c25768f", anthropic-ratelimit-input-tokens-limit:"16500000", anthropic-ratelimit-input-tokens-remaining:"16500000", anthropic-ratelimit-input-tokens-reset:"2025-12-01T13:54:31Z", anthropic-ratelimit-output-tokens-limit:"4250000", anthropic-ratelimit-output-tokens-remaining:"4250000", anthropic-ratelimit-output-tokens-reset:"2025-12-01T13:54:32Z", anthropic-rat

Congratulations, that's our first joke!

As you've noticed, this API doesn't look very convenient,
but that's the nature of low-level APIs.

Let's raise the abstraction level.
We'll create `*ChatOptions` to help us set and use parameters for our LLM provider requests:

In [5]:
val anthropicOptions = AnthropicChatOptions.builder()
    .model(AnthropicApi.ChatModel.CLAUDE_SONNET_4_0)
    .temperature(0.7)
    .maxTokens(1024)
    .build()
//val openAiOptions = OpenAiChatOptions.builder()
//    .model(OpenAiApi.ChatModel.CHATGPT_4_O_LATEST)
//    .temperature(0.7)
//    .build()

Now let's create a `*ChatModel`, which will help us get another joke:

In [6]:
val anthropicChat = AnthropicChatModel.builder()
    .anthropicApi(anthropicApi)
    .defaultOptions(anthropicOptions)
    .build()
//val openAiChat = OpenAiChatModel.builder()
//    .openAiApi(openAiApi)
//    .defaultOptions(openAiOptions)
//    .build()

Let's ask for a joke about Kotlin:

In [7]:
anthropicChat.call("Tell me a joke about Kotlin")
//openAiChat.call("Tell me a joke about Kotlin")

Here's a Kotlin joke for you:

Why did the Java developer switch to Kotlin?

Because they were tired of writing `NullPointerException` love letters to their code! 💕

(In Kotlin, null safety is built into the type system, so you're much less likely to encounter those pesky NPEs that Java developers know all too well!)

And now we have a joke about Kotlin!

This API is still specific to a particular provider.
You might need an even higher level of abstraction,
where your logic doesn't depend on which model you're using.

For this, we'll use the `ChatClient`:

In [8]:
val chatClient = ChatClient.create(anthropicChat)
chatClient.prompt("Tell me a joke about Kotlin").call().content()

Why did the Java developer switch to Kotlin?

Because they were tired of writing `public static void main(String[] args)` when they could just write `fun main()` and actually have fun! 

🎯 *Bonus punchline: They also got tired of null pointer exceptions ruining their day!*

Congratulations! Now we can write a simple joke and anecdote application.

Check out the next notebooks to learn more about Kotlin and Spring AI.